# Comet 追踪

有两种方式可以使用 Comet 追踪您的 LangChains 执行过程：

1. 将 `LANGCHAIN_COMET_TRACING` 环境变量设置为 "true"。这是推荐的方式。
2. 手动导入 `CometTracer` 并显式传递它。

In [ ]:
import os

import comet_llm
from langchain_openai import OpenAI

os.environ["LANGCHAIN_COMET_TRACING"] = "true"

# 如果未设置 API 密钥，连接到 Comet
comet_llm.init()

# comet 文档，使用环境变量配置 comet
# https://www.comet.com/docs/v2/api-and-sdk/llm-sdk/configuration/
# 这里我们配置 comet 项目
os.environ["COMET_PROJECT_NAME"] = "comet-example-langchain-tracing"

from langchain.agents import AgentType, initialize_agent, load_tools

In [ ]:
# 带追踪的 Agent 运行。确保已适当设置 OPENAI_API_KEY 以运行此示例。

llm = OpenAI(temperature=0)
tools = load_tools(["llm-math"], llm=llm)

In [ ]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent.run("2 的 .123243 次方是多少?")  # 这应该会被追踪
# 控制台中应该会打印出类似以下的链接 URL：
# https://www.comet.com/<workspace>/<project_name>
# 该 URL 可用于在 Comet 中查看 LLM 链。

In [ ]:
# 现在，我们取消设置环境变量并使用上下文管理器。
if "LANGCHAIN_COMET_TRACING" in os.environ:
    del os.environ["LANGCHAIN_COMET_TRACING"]

from langchain_community.callbacks.tracers.comet import CometTracer

tracer = CometTracer()

# 重新创建 LLM、工具和代理，并将回调传递给它们
llm = OpenAI(temperature=0)
tools = load_tools(["llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent.run(
    "2 的 .123243 次方是多少?", callbacks=[tracer]
)  # 这应该会被追踪